# Initialization

In [ ]:
# %load init.ipy
%reload_ext autoreload
%autoreload 2

import os, sys, logging, datetime, warnings, shutil
from importlib import reload

import numpy as np
import scipy as sp
import scipy.stats
import matplotlib as mpl
import matplotlib.pyplot as plt

import kalepy as kale
import kalepy.utils
import kalepy.plot
# The `nbshow` command runs `plt.show()` in interactive jupyter notebooks, but closes
#   figures when run from the command-line (notebooks are converted to scripts and run as tests)
from kalepy.plot import nbshow

import warnings
# warnings.simplefilter("error")   # WARNING: this is breaking jupyter at the moment (2021-02-14)

In [ ]:
DISTRIBUTION_CLASSES = kale.kernels._get_all_distribution_classes()

# Distributions

In [ ]:
def test_distribs():
    distribs = DISTRIBUTION_CLASSES
    num = len(distribs)
    xe, xc, dx = kale.utils.bins(-5.0, 5.0, 300)

    fig, axes = plt.subplots(figsize=[14, 4], ncols=num, sharey=True, sharex=True)
    for ax in axes:
        ax.grid(alpha=0.15)

    ax.set_xlim([-4.2, 4.2])

    for ax, dist in zip(axes, distribs):
        dist = dist()
        name = dist.name()
        print(name)
        ax.set_title(name)
        pdf = dist.evaluate(xc[np.newaxis, :], 1).squeeze()
        tot = np.trapz(pdf, x=xc)
        l1, = ax.plot(xc, pdf, 'b-')

        ax.plot(xc, np.cumsum(pdf*dx), 'r', ls=(0, [8, 4]), alpha=0.75)
        # ax.plot(xc, dist.cdf(xc), 'k', ls=(0, [1, 1]), lw=3.0, alpha=0.65)

        # Make sure that the total probability is near unity
        assert np.allclose(tot, 1.0, rtol=3e-2), "PDF is not unitary"
        # Make sure distributions have the right support
        if dist.FINITE:
            outside = (xc < -1.0) | (1.0 < xc)
            inside = (-1.0 < xc) & (xc < 1.0)
            assert np.allclose(pdf[outside], 0.0, atol=1e-6), "PDF does not have finite support"
        else:
            inside = slice(None)

        assert np.all(pdf[inside] > 0.0), "PDF is not non-zero over support"
        
    return

test_distribs()

## Check Support and Unitarity

In [ ]:
distribs = DISTRIBUTION_CLASSES
NUM = int(1e3)

num_dist = len(distribs)
fig, axes = plt.subplots(figsize=[6*num_dist, 4], ncols=num_dist, sharey=True, sharex=False)
for ax in axes:
    ax.grid(alpha=0.15)
    ax.set_xlim([-4, 4])

xe, xc, dx = kale.utils.bins(-5.0, 5.0, 50)
# ax.set_xlim([-4.2, 4.2])

fine = kale.utils.spacing(xe, 'lin', 300)

for ax, dist in zip(axes, distribs):
    dist = dist()
    name = dist.name()
    xlim = [-1.5, 1.5] if dist.FINITE else [-4.0, 4.0]
    ax.set_xlim(xlim)

    print(name)
    ax.set_title(name)
    pdf = dist.evaluate(fine[np.newaxis, :], 1).squeeze()
    l1, = ax.plot(fine, pdf, 'b-')
    tot = np.trapz(pdf, x=fine)
    print("tot = {:.4e}".format(tot))
    assert np.allclose(tot, 1.0, rtol=1e-2), "PDF is not unitary"

    samp = dist.sample(NUM)
    ax.hist(samp, bins=xe, density=True, edgecolor='b', facecolor='dodgerblue', alpha=0.5, rwidth=0.9)
    xvals = samp
    yvals = np.random.normal(-0.02, 0.01, xvals.size)
    ax.scatter(xvals, yvals, marker='.', s=2, alpha=0.02, color='b')

    # Make sure samples are in the right regions
    if dist.FINITE:
        # outside = ( < -1.0) | (1.0 < xc)
        inside = (-1.0 <= xvals) & (xvals <= 1.0)
    else:
        inside = np.ones_like(xvals, dtype=bool)

    assert np.all(inside), "samples are not within support"

    
# nbshow()

In [ ]:
distribs = DISTRIBUTION_CLASSES

NUM = int(1e3)
LEV = 4
bin_extr = [-5, 5]
ax_extr = [-3, 3]

num_dists = len(distribs)

# Setup Figure / Axes
# --------------------------------------
gs = mpl.gridspec.GridSpec(4, num_dists)
fig = plt.figure(figsize=[6*num_dists, 8])
axes = np.empty((2, num_dists), dtype=object)
for ii, jj in np.ndindex(axes.shape):
    xx = slice(0, 1, None) if ii == 0 else slice(1, 4, None)
    prev = axes[0, jj] if ii > 0 else None
    axes[ii, jj] = plt.subplot(gs[xx, jj], sharex=prev)
    axes[ii, jj].grid(alpha=0.15)
    
# Setup Grids / Bins
# -------------------------------
xe, xc, dx = kale.utils.bins(*bin_extr, 100)
ye, yc, dy = kale.utils.bins(*bin_extr, 102)

edges = [xe, ye]
# areas = dx[:, np.newaxis] * dy[np.newaxis, :]
# percs = sp.stats.norm.cdf([1.0, 2.0])
percs = sp.stats.norm.cdf([1.0])
xg, yg = np.meshgrid(xc, yc, indexing='ij')
grid = [xg.flatten(), yg.flatten()]

# Iterate over distributions and test
# -------------------------------------------------
for axcol, dist in zip(axes.T, distribs):
    dist = dist()
    name = dist.name()
    print("\n==== {} ====".format(name))
    # if not name.lower().startswith('tri'): continue
    
    pdf_e = dist.grid([xe, ye])
    pdf_c = dist.grid([xc, yc])

    pdf_e_xproj = np.trapz(pdf_e, x=ye, axis=-1)
    pdf_e_yproj = np.trapz(pdf_e, x=xe, axis=0)
    # print(np.trapz(pdf_e_xproj, x=xe))
    # print(np.trapz(pdf_e_yproj, x=ye))
    
    pmf = kale.utils.trapz_dens_to_mass(pdf_e, edges)
    samp = dist.sample(NUM, 2)
    
    ax = axcol[0]
    ax.set_title(name)
    
    ax.plot(xe, pdf_e_xproj, 'b-')
    xvals = samp[0]
    yvals = np.random.normal(-0.02, 0.01, xvals.size)
    ax.scatter(xvals, yvals, marker='.', s=2, alpha=0.03, color='b')
    
    ax = axcol[1]
    vals = pmf
    levels = np.unique(np.percentile(vals[vals > 0], 100*np.atleast_1d(percs)))
    levels = None
    
    ax.contour(xg, yg, vals, levels=levels, zorder=10, linewidths=4.0, colors='0.9', alpha=1.0)
    ax.contour(xg, yg, vals, levels=levels, zorder=11, linewidths=2.0, colors='blue')

    ax.scatter(*samp, facecolor='firebrick', edgecolor='0.25', alpha=0.04)
    
    ax.set(xlim=ax_extr)
    
    # Tests
    # -------------
    # Make sure that the total probability is near unity
    tot = np.sum(pmf)
    # print("tot = ", tot) # , kale.utils.stats_str(tot), kale.utils.array_str(tot))
    assert np.allclose(tot, 1.0, rtol=3e-2), "PDF is not unitary"
    # Make sure distributions have the right support
    if dist.FINITE:
        inside = dist.inside(grid)
        inside = inside.reshape(xg.shape)
        outside = ~inside
        assert np.allclose(pdf_c[outside], 0.0, atol=1e-6), "PDF does not have finite support"
    else:
        inside = slice(None)

    assert np.all(pdf_c[inside] > 0.0), "PDF is not non-zero over support"
    
    
# nbshow()

## Covariance

# Particular Kernels